In [2]:
"""
Generating ewt data for different cycles for all levels

N4 F1

"""

'\nGenerating ewt data for different cycles for all levels\n\nN4 F1\n\n'

In [3]:
import pandas as pd
import numpy as np
import pywt
import matplotlib.pyplot as plt
import os
import glob
import re

In [4]:
import numpy as np
#%EWT functions
def ewt_vineet(f,boundaries):
    # signal spectrum
    ff = np.fft.fft(f)
    ff = abs(ff[0:int(np.ceil(ff.size / 2))])  # one-sided magnitude

    #extend the signal by mirroring to deal with boundaries
    ltemp = int(np.ceil(f.size/2)) #to behave the same as matlab's round
    fMirr =  np.append(np.flip(f[0:ltemp-1],axis = 0),f)
    fMirr = np.append(fMirr,np.flip(f[-ltemp-1:-1],axis = 0))
    ffMirr = np.fft.fft(fMirr)
    # build the corresponding filter bank
    mfb = EWT_Meyer_FilterBank(boundaries, ffMirr.size)

    # filter the signal to extract each subband
    ewt = np.zeros(mfb.shape)
    for k in range(mfb.shape[1]):
        ewt[:, k] = np.real(np.fft.ifft(np.conjugate(mfb[:, k]) * ffMirr))
    ewt = ewt[ltemp - 1:-ltemp, :]

    return ewt


def EWT_Meyer_FilterBank(boundaries, Nsig):
    """
     =========================================================================
     function mfb=EWT_Meyer_FilterBank(boundaries,Nsig)

     This function generate the filter bank (scaling function + wavelets)
     corresponding to the provided set of frequency segments

     Input parameters:
       -boundaries: vector containing the boundaries of frequency segments (0
                    and pi must NOT be in this vector)
       -Nsig: signal length

     Output:
       -mfb: cell containing each filter (in the Fourier domain), the scaling
             function comes first and then the successive wavelets

     Author: Jerome Gilles
     Institution: UCLA - Department of Mathematics
     Year: 2012
     Version: 1.0

     Python Version: Vinícius Rezende Carvalho - vrcarva@ufmg.br
     Universidade Federal de Minas Gerais - Brasil
     Núcleo de Neurociências
     =========================================================================
     """
    Npic = len(boundaries)
    # compute gamma
    gamma = 1
    for k in range(Npic - 1):
        r = (boundaries[k + 1] - boundaries[k]) / (boundaries[k + 1] + boundaries[k])
        if r < gamma:
            gamma = r
    r = (np.pi - boundaries[Npic - 1]) / (np.pi + boundaries[Npic - 1])
    if r < gamma:
        gamma = r
    gamma = (1 - 1 / Nsig) * gamma  # this ensure that gamma is chosen as strictly less than the min

    mfb = np.zeros([Nsig, Npic + 1])

    # EWT_Meyer_Scaling
    Mi = int(np.floor(Nsig / 2))
    w = np.fft.fftshift(np.linspace(0, 2 * np.pi - 2 * np.pi / Nsig, num=Nsig))
    w[0:Mi] = -2 * np.pi + w[0:Mi]
    aw = abs(w)
    yms = np.zeros(Nsig)
    an = 1. / (2 * gamma * boundaries[0])
    pbn = (1. + gamma) * boundaries[0]
    mbn = (1. - gamma) * boundaries[0]
    for k in range(Nsig):
        if aw[k] <= mbn:
            yms[k] = 1
        elif ((aw[k] >= mbn) and (aw[k] <= pbn)):
            yms[k] = np.cos(np.pi * EWT_beta(an * (aw[k] - mbn)) / 2)
    yms = np.fft.ifftshift(yms)
    mfb[:, 0] = yms

    # generate rest of the wavelets
    for k in range(Npic - 1):
        mfb[:, k + 1] = EWT_Meyer_Wavelet(boundaries[k], boundaries[k + 1], gamma, Nsig)

    mfb[:, Npic] = EWT_Meyer_Wavelet(boundaries[Npic - 1], np.pi, gamma, Nsig)

    return mfb
def EWT_beta(x):
    """
    Beta = EWT_beta(x)
    function used in the construction of Meyer's wavelet
    """
    if x<0:
        bm=0
    elif x>1:
        bm=1
    else:
        bm=(x**4)*(35.-84.*x+70.*(x**2)-20.*(x**3))
    return bm
def EWT_Meyer_Wavelet(wn,wm,gamma,Nsig):
    """
    =========================================================
    ymw=EWT_Meyer_Wavelet(wn,wm,gamma,N)
    
    Generate the 1D Meyer wavelet in the Fourier
    domain associated to scale segment [wn,wm] 
    with transition ratio gamma
    
    Input parameters:
      -wn : lower boundary
      -wm : upper boundary
      -gamma : transition ratio
      -N : number of point in the vector
    
    Output:
      -ymw: Fourier transform of the wavelet on the band [wn,wm]
    Author: Jerome Gilles
    Institution: UCLA - Department of Mathematics
    Year: 2012
    Version: 1.0
    
    Python Version: Vinícius Rezende Carvalho - vrcarva@ufmg.br
    Universidade Federal de Minas Gerais - Brasil
    Núcleo de Neurociências 
    ==========================================================            
    """
    Mi=int(np.floor(Nsig/2))
    w=np.fft.fftshift(np.linspace(0,2*np.pi - 2*np.pi/Nsig,num = Nsig))
    w[0:Mi]=-2*np.pi+w[0:Mi]
    aw=abs(w)
    ymw=np.zeros(Nsig)
    an=1./(2*gamma*wn)
    am=1./(2*gamma*wm)
    pbn=(1.+gamma)*wn
    mbn=(1.-gamma)*wn
    pbm=(1.+gamma)*wm
    mbm=(1.-gamma)*wm

    for k in range(Nsig):
        if ((aw[k]>=pbn) and (aw[k]<=mbm)):
            ymw[k]=1
        elif ((aw[k]>=mbm) and (aw[k]<=pbm)):
            ymw[k]=np.cos(np.pi*EWT_beta(am*(aw[k]-mbm))/2)
        elif ((aw[k]>=mbn) and (aw[k]<=pbn)):
            ymw[k]=np.sin(np.pi*EWT_beta(an*(aw[k]-mbn))/2)
    ymw=np.fft.ifftshift(ymw)
    return ymw

In [5]:
pwd

'C:\\Users\\vinee\\Documents\\BTP'

In [6]:
os.chdir('resampled_dataset_f1_n4/')

In [8]:
my_files = glob.glob('*txt')

In [9]:
len(my_files)

93000

In [10]:
idx = []

In [11]:
for i in range(len(my_files)):
    string1 = my_files[i]
    p = list(map(int, re.findall(r'\d+', string1)))
    if p[-2] == 1 and p[-3] == 1:
        idx.append(i)

In [12]:
len(idx)

15

In [16]:
my_files[idx[1]]

'F1_raw_N4_R1_cycle_1_30.txt'

In [17]:
idx = []

In [18]:
for i in range(len(my_files)):
    string1 = my_files[i]
    p = list(map(int, re.findall(r'\d+', string1)))
    if p[-2] == 1 and p[-3] == 1:
        idx.append(i)

In [19]:
len(idx)

15

In [20]:
idx = []
for i in range(len(my_files)):
    string1 = my_files[i]
    p = list(map(int, re.findall(r'\d+', string1)))
    if p[-2] == 1 and p[-3] == 1:
        idx.append(i)

In [21]:
len(idx)

15

In [22]:
idx = []
for i in range(len(my_files)):
    string1 = my_files[i]
    p = list(map(int, re.findall(r'\d+', string1)))
    if p[-2] == 1 and p[-3] == 1:
        idx.append(i)

In [23]:
len(idx)

15

In [24]:
loudness_data_n4 = np.array([-0.161520,0.300446,0.502216,0.559076,0.654683,0.955243,0.968337,0.885020,0.965291,1.290606,1.397530,1.460671,1.729242,1.781090])


In [25]:
boundaries = np.loadtxt('C:/users/vinee/Documents/BTP/n4_avg/boundaries_n4.txt')

In [26]:
boundaries

array([0.84317118, 1.30193917, 1.90199771, 2.25771169, 2.4654886 ,
       2.68656333, 2.8045807 , 2.92259809, 3.04061548])

In [39]:
"""
Generation of Dataset(F1N4) using R1 1000 cycles
"""

'\nGeneration of Dataset(F1N4) using R1 1000 cycles\n'

In [46]:
x_train = []
y_train = []

In [47]:
for j in range(1,1001,1):
    idx = []
    for i in range(len(my_files)):
        string1 = my_files[i]
        p = list(map(int, re.findall(r'\d+', string1)))
        if p[-2] == j and p[-3] == 1:
            idx.append(i)
    res = np.zeros((14,10,126))
    i  = 0
    for m in idx:
        if my_files[m][-6:-4]=='30':
            continue
        file = np.loadtxt(my_files[m])
        ewt = ewt_vineet(file,boundaries)
        ewt = ewt.transpose()
        res[i] = ewt
        i+=1
    x_train.append(res)
    y_train.append(loudness_data_n4)

In [48]:
len(x_train)

1000

In [49]:
x = np.array(x_train)

In [50]:
y = np.array(y_train)

In [51]:
len(y_train)

1000

In [52]:
x.shape

(1000, 14, 10, 126)

In [53]:
np.save('C:/Users/vinee/Documents/BTP/ewt_x_n4_cnn_1',x)

In [54]:
np.save('C:/Users/vinee/Documents/BTP/ewt_y_n4_cnn_1',y)

In [55]:
for j in range(1,1001,1):
    idx = []
    for i in range(len(my_files)):
        string1 = my_files[i]
        p = list(map(int, re.findall(r'\d+', string1)))
        if p[-2] == j and p[-3] == 2:
            idx.append(i)
    res = np.zeros((14,10,126))
    i  = 0
    for m in idx:
        if my_files[m][-6:-4]=='30':
            continue
        file = np.loadtxt(my_files[m])
        ewt = ewt_vineet(file,boundaries)
        ewt = ewt.transpose()
        res[i] = ewt
        i+=1
    x_train.append(res)
    y_train.append(loudness_data_n4)

In [56]:
x = np.array(x_train)

In [57]:
y = np.array(y_train)

In [58]:
y.shape

(2000, 14)

In [59]:
np.save('C:/Users/vinee/Documents/BTP/ewt_x_n4_cnn_2',x)

In [60]:
np.save('C:/Users/vinee/Documents/BTP/ewt_y_n4_cnn_2',y)

In [ ]:
for j in range(1,1001,1):
    idx = []
    for i in range(len(my_files)):
        string1 = my_files[i]
        p = list(map(int, re.findall(r'\d+', string1)))
        if p[-2] == j and p[-3] == 3:
            idx.append(i)
    res = np.zeros((14,10,126))
    i  = 0
    for m in idx:
        if my_files[m][-6:-4]=='30':
            continue
        file = np.loadtxt(my_files[m])
        ewt = ewt_vineet(file,boundaries)
        ewt = ewt.transpose()
        res[i] = ewt
        i+=1
    x_train.append(res)
    y_train.append(loudness_data_n4)

In [ ]:
x = np.array(x_train)
y = np.array(y_train)

In [ ]:
np.save('C:/Users/vinee/Documents/BTP/ewt_y_n4_cnn_3',y)
np.save('C:/Users/vinee/Documents/BTP/ewt_x_n4_cnn_3',x)